In [1]:
from test_beta_package import *
%matplotlib inline

In [2]:
path_data_folder, path_raw_data_folder, path_process_data_folder, path_top_folder, path_info_dir, path_analysis_dir, \
path_plots_dir, path_single_pred_dir, path_multi_pred_dir = folder_structure()

In [3]:
data_continous_ratemaps, data_binned_glm = \
    processing_and_loading(path_raw_data_folder, path_process_data_folder,  # mat_file="diana_chasing2_notreheaded.mat")
                           data_continous_ratemaps_pickle="rm_pre_data_diana_chasing2_notreheaded_XYZeuler_notricks_eo.pkl",
                           data_binned_glm_pickle="binned_4_GLM_rm_pre_data_diana_chasing2_notreheaded_XYZeuler_notricks_eo.pkl")


In [4]:
predictors_name_list, cells_id, animal_name = get_data_information(path_info_dir, data_continous_ratemaps)

In [5]:
dict_param = dict_parameters_hmm(path_info_dir, animal_name, num_dimen=1, num_categ_obs=2, N_iters=100,
                                 tolerance=10**-5, num_indep_neurons=2, num_predictors=8, max_num_states=3,
                                 observation_type="input_driven_obs",
                                 transistion_type="sticky", optim_method="em")

In [6]:
cell_index, tot_time = cells_selection_manual(path_info_dir, data_continous_ratemaps, data_binned_glm, dict_param, 'cells_selection.csv')

In [7]:
# processing of data and creation of istances for the inference
glmhmms_ista, process_neur, inputs_list, T_list, tot_masked_indices_list, path_plots_list, plots_folder = \
    data_structure(path_info_dir, path_analysis_dir, path_single_pred_dir, data_continous_ratemaps, data_binned_glm,
                   dict_param,
                   tot_time, cell_index, predictors_name_list, predictor_file="test_predictors.csv")

NAME OF THE FOLDER IS C:\Users\teof\PycharmProjects\pythonREGITglmhmm\postural_glm_hmm\results_folder\plots_dir/single_pred_dir/diana_notreheaded_singlepredictor_2022-10-24_run/
IMPORTANT: given the presence of nans (missing points of the camera) part of the data are deleted.If the number of missing points is 'small enough', it should not interfere with the inference.Below the ratio of missing points
the input list is [[array([[10.6933533 ,  1.        ],
       [10.94194185,  1.        ],
       [11.20825033,  1.        ],
       ...,
       [ 0.37459429,  1.        ],
       [ 0.50266247,  1.        ],
       [ 0.64292032,  1.        ]]), array([[10.6933533 ,  1.        ],
       [10.94194185,  1.        ],
       [11.20825033,  1.        ],
       ...,
       [ 0.37459429,  1.        ],
       [ 0.50266247,  1.        ],
       [ 0.64292032,  1.        ]])]]
the input list is [[array([[10.6933533 ,  1.        ],
       [10.94194185,  1.        ],
       [11.20825033,  1.        ],
  

In [12]:
fit_ll_states_list, glmhmms_ista, time_states_comp = inference_section(path_analysis_dir, path_info_dir, dict_param,
                                                                       glmhmms_ista=glmhmms_ista, process_neur=process_neur,
                                                                       inputs_list=inputs_list)

0


  0%|          | 0/100 [00:00<?, ?it/s]

computation time loop 0 is 9.713443279266357
1


  0%|          | 0/100 [00:00<?, ?it/s]

computation time loop 1 is 14.18449068069458
2


...

computation time loop 13 is 16.080402374267578
6


  0%|          | 0/100 [00:00<?, ?it/s]

computation time loop 14 is 13.231858253479004
7


  0%|          | 0/100 [00:00<?, ?it/s]

computation time loop 15 is 15.940877437591553
The total time for the inference is 243.55352592468262
the input list after inference is (73132, 2)


In [ ]:
posterior_probs_list = posterior_prob_process(path_info_dir, path_analysis_dir, dict_param=dict_param,
                                              glmhmms_ista=glmhmms_ista, process_neur=process_neur,
                                              inputs_list=inputs_list)

In [ ]:
states_occupancies = states_occupancies_computation(path_analysis_dir, posterior_probs_list)

In [ ]:
log_like_evolution_per_states(path_analysis_dir, path_info_dir, dict_objects=0, dict_processed_objects=0,
                              dictionary_information=0)

In [ ]:
states_occupancies_histogram(path_analysis_dir, path_info_dir, dict_param=dict_param,
                             states_occupancies=states_occupancies)

In [ ]:
transition_prob_matrix(path_analysis_dir, path_info_dir, glmhmms_ista=glmhmms_ista, dict_param=dict_param)

In [ ]:
weights_distribution_histogram(path_analysis_dir, path_info_dir, animal_name=animal_name)